# Mapping District for Price Prediction


In [20]:
df['date'].head()

0   2008-10-01
1   2009-01-01
2   2009-04-01
3   2009-07-01
4   2009-10-01
Name: date, dtype: datetime64[ns]

In [9]:
#taken from another file

import pandas as pd
from sklearn.ensemble import RandomForestRegressor


df['date_numeric'] = (df['date'] - df['date'].min())  / pd.Timedelta(1, unit='D')

df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')

df = df.dropna()

X = df[['latitude', 'longitude', 'date_numeric']]
y = df['price_sqm']


model = RandomForestRegressor(n_estimators=100)
model.fit(X, y)

# Make predictions for the entire dataset
df['predicted_price_sqm'] = model.predict(X)

# Make a prediction for the next year
next_year = df['date'].max() + pd.DateOffset(years=1)
next_year_numeric = (next_year - df['date'].min())  / pd.Timedelta(1, unit='D')

# Make a prediction for each district
districts = df['district'].unique()

for district in districts:
    district_data = df[df['district'] == district]
    
    # Use the mean latitude and longitude for the district
    latitude = district_data['latitude'].mean()
    longitude = district_data['longitude'].mean()
    
    predicted_price_next_year = model.predict([[latitude, longitude, next_year_numeric]])
    
    print(f"Predicted price per sqm for district {district} in the next year: {predicted_price_next_year[0]}")



/Users/lynndalee/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning:

X does not have valid feature names, but RandomForestRegressor was fitted with feature names

/Users/lynndalee/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning:

X does not have valid feature names, but RandomForestRegressor was fitted with feature names

/Users/lynndalee/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning:

X does not have valid feature names, but RandomForestRegressor was fitted with feature names

/Users/lynndalee/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning:

X does not have valid feature names, but RandomForestRegressor was fitted with feature names

/Users/lynndalee/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning:

X does not have valid feature names, but RandomForestRegressor was fitted with feature names

/Users/lynndalee/opt/anaconda3/lib/python3.9/site-packages/s

Predicted price per sqm for district Bang Kapi in the next year: 71753.89
Predicted price per sqm for district Bang Khae in the next year: 62305.83
Predicted price per sqm for district Bang Khen in the next year: 36892.28
Predicted price per sqm for district Bang Kho Laem in the next year: 64231.32
Predicted price per sqm for district Bang Khun Thian in the next year: 58072.48
Predicted price per sqm for district Bang Na in the next year: 72926.94
Predicted price per sqm for district Watthana in the next year: 166675.6
Predicted price per sqm for district Yan Nawa in the next year: 84304.97
Predicted price per sqm for district Bang Phlat in the next year: 88924.97
Predicted price per sqm for district Bang Rak in the next year: 313679.83
Predicted price per sqm for district Bang Sue in the next year: 135668.0
Predicted price per sqm for district Bangkok Noi in the next year: 64266.78
Predicted price per sqm for district Bangkok Yai in the next year: 62500.0
Predicted price per sqm for d

/Users/lynndalee/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning:

X does not have valid feature names, but RandomForestRegressor was fitted with feature names

/Users/lynndalee/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning:

X does not have valid feature names, but RandomForestRegressor was fitted with feature names

/Users/lynndalee/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning:

X does not have valid feature names, but RandomForestRegressor was fitted with feature names

/Users/lynndalee/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning:

X does not have valid feature names, but RandomForestRegressor was fitted with feature names

/Users/lynndalee/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning:

X does not have valid feature names, but RandomForestRegressor was fitted with feature names

/Users/lynndalee/opt/anaconda3/lib/python3.9/site-packages/s

In [22]:
import numpy as np
import plotly.express as px

condominiums = df['name'].unique()
predictions = []
prediction_ranges = []

for condominium in condominiums:
    condominium_data = df[df['name'] == condominium]
    
    latitude = condominium_data['latitude'].mean()
    longitude = condominium_data['longitude'].mean()

    individual_tree_predictions = np.array([tree.predict([[latitude, longitude, next_year_numeric]]) for tree in model.estimators_])
    predicted_price_next_year = individual_tree_predictions.mean()
    prediction_std_dev = individual_tree_predictions.std()
    
    prediction_range = [predicted_price_next_year - prediction_std_dev, predicted_price_next_year + prediction_std_dev]
    
    predictions.append(predicted_price_next_year)
    prediction_ranges.append(prediction_range)


In [23]:

    
predicted_df = pd.DataFrame({
    'name': condominiums,
    'latitude': df.groupby('name')['latitude'].mean().values,
    'longitude': df.groupby('name')['longitude'].mean().values,
    'predicted_price_sqm_next_year': predictions,
    'prediction_range': prediction_ranges,
    'district': df.groupby('name')['district'].first().values
})

# Create a heatmap
fig = px.density_mapbox(predicted_df, lat='latitude', lon='longitude', z='predicted_price_sqm_next_year', 
                        radius=10, center=dict(lat=0, lon=180), zoom=0, 
                        mapbox_style="stamen-terrain", title='Predicted Prices Heatmap', hover_data=['district','name'])
fig.show()